# 0) 50 shades of binary search

### i) Any matching index, or -1

Return index of any matching value in array, or -1.

*Note 1*: The while loop continues even when ```left == right```. Example:

Say we look for 7:

... - 3 - 7 - ...  
... - L - R - ...

In the above configuration, `mid` becomes `left` because the int divison rounds down. It's only at the next step of the while loop that `mid` can become `right`.

*Note 2*: the `left` and`right` pointers are bound to cross because either one of them moves at each step of the while loop.

In [1]:
# Basic
# Return index of any matching value in array, or -1

from typing import List, Tuple

def binary_search(array: List[int], target: int) -> int:
    left, right = 0, len(array)-1

    while left <= right:
        mid = (left + right) // 2 # No overflow in Python
        if array[mid] > target:
            right = mid - 1
        elif array[mid] < target:
            left = mid + 1
        else:
            return mid
    return -1

### ii) Duplicates: leftmost/righmost matching index, or -1

In [2]:
# Duplicates

def binary_search_tips(array: List[int], target: int, type : str = 'leftmost') -> int:
    left, right, index = 0, len(array)-1, -1

    while left <= right:
        mid = (left + right) // 2 # No overflow in Python
        if array[mid] > target:
            right = mid - 1
        elif array[mid] < target:
            left = mid + 1
        else:
            index = mid
            if type == 'leftmost':
                right = mid - 1
            if type == 'rightmost':
                left = mid + 1
    return index

def binary_search_duplicates(array: List[int], target: int) -> Tuple[int]:
    return binary_search_tips(array, target, 'leftmost'), binary_search_tips(array, target, 'rightmost')

### iii) Bisect: a more general approach

Bisect left (resp. right): returns the leftmost (resp. rightmost) index of matching value(s), or the index where the target value should be inserted.

Bisect returns outer indexes instead of the middle index. The outer indexes become equal in the last step of the while loop, since their difference decreases by at least 1 unit at each step:
* First `if` clause: the left index is shifted by 1 unit or more, since `left <= mid`
* Second `if` clause: `mid < right` since int division `//` rounds down in Python

Bisect left returns $min \{ i, array[i] \geq target \}$

Justification of the `if` clauses:
* `array[mid] < target`: `mid` cannot be the seeked index, since we look for the 'first' element $\geq$ target
* `else` condition i.e `array[mid] >= target`: `mid` could be the index we're looking for. Keep it for now and check elements on the left part of the array.

[Alternative implementation](https://gist.github.com/py-in-the-sky/e84fd9fc4db0da3f351631ba04b2d91b)

In [3]:
def bisect_left(array: List[int], target: int) -> int:
    left, right = 0, len(array)
    while left < right: # Strict inequality because `left == right` in the last step is guaranteed
        mid = (left + right) // 2
        if array[mid] < target:
            left = mid + 1 # `mid` cannot be the correct index, shift `left`` on its right
        else:
            right = mid # `array[mid] <= left`, so it could be to correct index
    return left # `right` works too

Bisect right returns $min \{ i, array[i] > target \}$, so it is just about turning the strict inequality into a large one:

In [4]:

def bisect_right(array: List[int], target: int) -> int:
    left, right = 0, len(array)
    while left < right:
        mid = (left + right) // 2
        if array[mid] <= target:
            left = mid + 1
        else:
            right = mid
    return left

def binary_insert(array: List[int], target: int) -> None:
    array.insert(bisect_left(array, target), target)
    return

def bisect_search(array: List[int], target: int, type : str = 'leftmost') -> int:
    index = bisect_left(array, target)
    if index < len(array) and array[index] == target:
        return index
    else:
        return -1

**Note that the right pointer is initialized as `len(array)`, not `len(array) - 1`**. This allows the returned value to be `len(array)`, in case it should be used for appending a value at the end of the array.

### 3-pointer implementations (l, r, m) vs 2-pointer implementations

| | 3-pointer | 2-pointer |
|----------|----------|----------|
| Bounds | r = len(array) - 1 | r = len(array) |
| While condition | l <= r | l < r |
| Shift | m = r + 1 | r = m |

Better use variations of `bisect_left` and `bisect_right` in most cases. 

In [1]:
def bisect_left(nums: list[int], target: int) -> int:
    l, r = 0, len(nums)
    while l < r:
        m = (l + r) // 2
        if nums[m] < target:
            l = m + 1
        else:
            r = m
    return l

def bisect_right(nums: list[int], target: int) -> int:
    l, r = 0, len(nums)
    while l < r:
        m = (l + r) // 2
        if nums[m] <= target:
            l = m + 1
        else:
            r = m
    return l

tests = {
    "Not there - Front": [20],
    "Not there - Middle": [1, 3, 9],
    "Not there - Tail": [1, 3, 4, 4, 5],
    "There - Unique": [1, 3, 4, 4, 7, 8, 9],
    "There - Multiple": [1, 3, 4, 4, 7, 7, 7, 7, 9]
}

for key, nums in tests.items():
    print(f"Left - {key} | {nums} | {bisect_left(nums, 7)}")
    print(f"Right - {key} | {nums} | {bisect_right(nums, 7)}", end="\n\n")

Left - Not there - Front | [20] | 0
Right - Not there - Front | [20] | 0

Left - Not there - Middle | [1, 3, 9] | 2
Right - Not there - Middle | [1, 3, 9] | 2

Left - Not there - Tail | [1, 3, 4, 4, 5] | 5
Right - Not there - Tail | [1, 3, 4, 4, 5] | 5

Left - There - Unique | [1, 3, 4, 4, 7, 8, 9] | 4
Right - There - Unique | [1, 3, 4, 4, 7, 8, 9] | 5

Left - There - Multiple | [1, 3, 4, 4, 7, 7, 7, 7, 9] | 4
Right - There - Multiple | [1, 3, 4, 4, 7, 7, 7, 7, 9] | 8

